In [1]:
# https://www.tensorflow.org/tutorials/keras/regression

from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import tensorflow as tf

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
import os


1.14.0


Collect data

In [3]:
column_names = ['fileId', 'row','rising_idx','falling_idx','volume','rising_weight',
                'falling_weight', 'first_val', 'last_val', 'delay']
def filterBad(dataset):
    initial_len = len(dataset)
    dataset = dataset[(dataset['rising_idx'] != 0)]
    dataset = dataset[(dataset['falling_idx'] != 0)]
    dataset = dataset[(dataset['volume'] > 500)]
#     dataset = dataset[(dataset['first_val'] < 30)]
#     dataset = dataset[(dataset['last_val'] < 30)]
    final_len = len(dataset)
    print('Rejected %d points (%f%%)' % ((initial_len-final_len), (initial_len-final_len)/initial_len))
    return dataset

raw_dataset =  pd.read_feather('../preprocessing/processed.feather')
raw_dataset = filterBad(raw_dataset)
# visualize_dataset = raw_dataset.sample(frac=0.01)
# sns.pairplot(visualize_dataset[column_names], diag_kind="kde")


dataset = raw_dataset.copy()
dataset.tail()

dataset = dataset.dropna()
dataset = dataset.drop(columns=['fileId', 'row'])
dataset = pd.get_dummies(dataset, prefix='', prefix_sep='')
# dataset.tail()
# train_dataset = dataset.sample(frac=0.8,random_state=0)
# test_dataset = dataset.drop(train_dataset.index)

# train_stats = train_dataset.describe()
# train_stats.pop("delay")
# train_stats = train_stats.transpose()
# train_stats
# train_labels = train_dataset.pop('delay')
# test_labels = test_dataset.pop('delay')







Rejected 1810263 points (0.166070%)


In [4]:
from sklearn.model_selection import train_test_split

X, y = dataset.iloc[:,:-1],dataset.iloc[:,-1]

train_dataset, test_dataset, train_labels, test_labels = train_test_split(X, y, test_size=0.2, random_state=123)

train_stats = train_dataset.describe().transpose()

1665171    -1888.38000
10469887     370.92400
1593196     1147.83000
4213913      234.00100
4896645    -1488.40000
10644609   -1023.21000
10275321   -1046.29000
5470257       -3.40788
6999509    -1452.61000
9164833      722.87200
4430505     -811.18000
4376785     2103.34000
7717261      189.44400
7104372      983.77300
5449251    -1841.02000
7269915      134.38300
505758       208.76400
6219525     -204.73300
3733296      821.54200
5839927     -200.84000
8614007     1538.04000
6066779      919.96000
9543783    -1215.13000
4277422    -1465.58000
5514181     1145.23000
4789881     -463.22900
3552         491.64700
2701117     1245.07000
5544331      734.08200
4261225      121.73900
               ...    
8015268      543.87300
10167185     157.47600
2408811    -1898.16000
5725746     -141.92200
2856693    -2485.93000
579217     -1557.97000
732694     -3340.65000
8096757    -1902.52000
9288148     2400.33000
3728489     -242.63800
4891079     -460.45100
3732963       66.42990
2823614    

In [24]:
L1_DIM = 64

In [6]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']

normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)


print(train_dataset)
print(normed_train_data)
print(train_stats['mean'])
def build_model():
  model = keras.Sequential([
    layers.Dense(L1_DIM, activation='relu', input_shape=[len(train_dataset.keys())]),
#     layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

model = build_model()

          rising_idx  falling_idx    volume  rising_weight  falling_weight  \
4389278          145          324    397895      933.50400     -1044.51000   
5703301          333          551    309988      740.38500      -690.48500   
2133620          230          427     28632       84.09030       -91.88660   
7775485          157          341     35077      103.68900       -86.14610   
9028950          139          318    136806      378.40500      -483.93800   
7303294          195          385    200376      614.59100      -630.45500   
1863732          598          874     16353       20.75600       -27.06440   
3202291          181          383  10138581    24487.30000    -17317.90000   
95605            168          353    877594     2492.53000     -2943.66000   
10591325         379          606     38297       81.47080       -98.73130   
6463261          258          461    155648      402.19300      -344.56000   
516954           368          593    992140     2111.44000     -

In [8]:

model.summary()

# example_batch = normed_train_data[:10]
# example_result = model.predict(example_batch)
# example_result

EPOCHS = 15

# filepath="ckpoints.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
# callbacks_list = [checkpoint]

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[tfdocs.modeling.EpochDots()])
#     callbacks = callbacks_list)

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                512       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 577
Trainable params: 577
Non-trainable params: 0
_________________________________________________________________

Epoch: 0, loss:5269.2334,  mean_absolute_error:23.8322,  mean_squared_error:5269.2305,  val_loss:3879.7772,  val_mean_absolute_error:16.8007,  val_mean_squared_error:3879.7534,  
........

KeyboardInterrupt: 

In [9]:

# tf.keras.utils.plot_model(model, to_file='model.png')

# plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)


# loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)
preds = model.predict(normed_test_data)
rmse = mean_squared_error(test_labels, preds)
rmae = mean_absolute_error(test_labels, preds)
r2 = r2_score(test_labels, preds)
print("MSE, MAE, r2: %f %f %f" % (rmse, rmae, r2))


MSE, MAE, r2: 1848.124631 11.616834 0.998699


In [33]:
i = 0

print('val mean = FULLFEATURES(%s)' % ','.join([str(train_stats['mean'][x]) for x in column_names[2:-1]]))
#activation(dot(input, kernel) + bias)
for layer in model.get_weights():
    s = 'W' if (i % 2 == 0) else 'B'
    vals = []
    for j,x in enumerate(layer.flatten()):
        if (i % 2 == 0):
            ft = column_names[2:-1][int(j / L1_DIM)]
            scaled = x / train_stats['std'][ft]
            vals.append('%f.to[T]' % scaled)
        else:
            vals.append('%f.to[T]' % x)
    print('val l%d%s = Seq(%s)' % (1+i//2, s, ','.join(vals)))
#     print('Layer %d' % i)
#     print(layer)
    i =  i + 1

val mean = FULLFEATURES(268.2145561837097,468.8421516397134,1883737.9202614257,3549.49099375219,-3594.0790894527363,895.8266651559064,88.2579832973909)
val l1W = Seq(0.017414.to[T],0.043573.to[T],0.001617.to[T],0.063180.to[T],0.010253.to[T],-0.023324.to[T],0.008253.to[T],-0.099895.to[T],0.042064.to[T],0.019148.to[T],0.054045.to[T],-0.076539.to[T],0.009661.to[T],-0.030295.to[T],0.011007.to[T],0.014330.to[T],-0.033842.to[T],-0.022774.to[T],-0.054721.to[T],0.038924.to[T],-0.035612.to[T],-0.000166.to[T],0.075991.to[T],0.028132.to[T],0.020888.to[T],0.036434.to[T],-0.117659.to[T],0.042636.to[T],-0.056284.to[T],-0.075248.to[T],-0.057049.to[T],-0.101635.to[T],-0.038393.to[T],0.010143.to[T],0.044220.to[T],0.012465.to[T],0.005401.to[T],0.009410.to[T],0.003110.to[T],0.061756.to[T],0.014013.to[T],0.010328.to[T],0.048747.to[T],0.029928.to[T],-0.030564.to[T],0.068709.to[T],-0.108280.to[T],-0.077954.to[T],-0.047952.to[T],0.051305.to[T],0.003847.to[T],-0.035289.to[T],0.000023.to[T],0.024519.to[T],0.04

In [36]:
# df2 = xgb.DMatrix(pd.DataFrame({"rising_idx":[157],
#     "falling_idx":[341],
#     "volume":[12039],
#     "rising_weight":[35.2498],
#     "falling_weight":[-28.1037]}))
df2 = pd.DataFrame({"rising_idx":[157],
    "falling_idx":[341],
    "volume":[12039],
    "rising_weight":[35.2498],
    "falling_weight":[-28.1037], 
    "first_val":[12],
    "last_val":[0]})
print(model.predict(norm(df2)))
print('True label = %f' % 509.093)

[[497.21515]]
True label = 509.093000


In [40]:
hist

,loss,mean_absolute_error,mean_squared_error,val_loss,val_mean_absolute_error,val_mean_squared_error,epoch
0,1960.505538,11.085601,1960.495850,1977.193724,10.932296,1977.186646,0
1,1909.706727,10.885015,1909.700684,1915.721155,10.794977,1915.719360,1
2,1828.697132,10.612532,1828.684204,1853.169155,10.450208,1853.172729,2
3,1744.994297,10.301882,1744.984497,1744.437534,10.109297,1744.441040,3
4,1672.969164,10.100968,1672.969727,1719.620754,10.042290,1719.632080,4
5,1642.198482,10.008043,1642.208374,1687.393910,9.925940,1687.396729,5
6,1618.648786,9.920607,1618.653564,1645.547099,9.809487,1645.541016,6
7,1600.134799,9.862710,1600.137573,1656.181159,9.924092,1656.178833,7
8,1580.953250,9.798059,1580.948364,1629.672884,9.983445,1629.671753,8
9,1564.923712,9.765280,1564.912354,1617.099227,9.930725,1617.101196,9


In [ ]:

model.summary()

# example_batch = normed_train_data[:10]
# example_result = model.predict(example_batch)
# example_result

EPOCHS = 40

# filepath="ckpoints.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
# callbacks_list = [checkpoint]

history = model.fit(
  train_dataset, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[tfdocs.modeling.EpochDots()])
#     callbacks = callbacks_list)

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()



Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 64)                512       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 577
Trainable params: 577
Non-trainable params: 0
_________________________________________________________________

Epoch: 0, loss:16776144114472384.0000,  mean_absolute_error:4861550.5000,  mean_squared_error:16763241250160640.0000,  val_loss:513169574993.0494,  val_mean_absolute_error:111827.4062,  val_mean_squared_error:513171947520.0000,  
...............

In [ ]:

# tf.keras.utils.plot_model(model, to_file='model.png')

# plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)


# loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)
preds = model.predict(test_dataset)
rmse = mean_squared_error(test_labels, preds)
rmae = mean_absolute_error(test_labels, preds)
r2 = r2_score(test_labels, preds)
print("MSE, MAE, r2: %f %f %f" % (rmse, rmae, r2))


In [ ]:
i = 0
#activation(dot(input, kernel) + bias)
for layer in model.get_weights():
    s = 'W' if (i % 2 == 0) else 'B'
    print('val l%d%s = Seq(%s)' % (1+i//2, s, ','.join(['%f.to[T]' % x for x in layer.flatten()])))
#     print('Layer %d' % i)
#     print(layer)
    i =  i + 1